In [5]:
import mxnet as mx
import numpy as np
import time
import pandas as pd

import cv2

import matplotlib.pyplot as plt
%matplotlib inline
path='http://data.mxnet.io/models/imagenet-11k/'
[mx.test_utils.download(path+'resnet-152/resnet-152-symbol.json', dirname='models'),
 mx.test_utils.download(path+'resnet-152/resnet-152-0000.params', dirname='models'),
 mx.test_utils.download(path+'synset.txt', dirname='models')]

['models/resnet-152-symbol.json',
 'models/resnet-152-0000.params',
 'models/synset.txt']

In [7]:
sym, arg_params, aux_params = mx.model.load_checkpoint('models/resnet-152', 0)
fc = sym.get_children()[0]
mod = mx.mod.Module(symbol=fc, context=mx.cpu(), label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,200,200))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)

In [8]:
data0 = pd.read_csv('fdata/fdata.csv', names=['name','state'])

In [9]:
num_class = len(data0['state'].unique())
ges_to_num = dict({(g,i) for i, g in enumerate(data0['state'].unique())})
num_to_ges = dict({(i,g) for i, g in enumerate(data0['state'].unique())})
num_class, ges_to_num

(26,
 {' *': 23,
  ' +': 21,
  ' -': 22,
  ' /': 24,
  ' 0': 20,
  ' 1': 11,
  ' 2': 12,
  ' 3': 13,
  ' 4': 14,
  ' 5': 15,
  ' 6': 16,
  ' 7': 17,
  ' 8': 18,
  ' 9': 19,
  ' =': 25,
  ' alphabet1': 3,
  ' alphabet2': 4,
  ' alt': 10,
  ' ctrl': 9,
  ' initial': 0,
  ' number': 2,
  ' ready': 1,
  ' shift': 8,
  ' special1': 5,
  ' special2': 6,
  ' special3': 7})

In [10]:
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])


prefix = 'fdata/hand_pic_v2/'


out = []
labels = []

for i, (im_name, state) in enumerate(data0.values):
    print i
    im_path = prefix + im_name
    
    
    img = cv2.imread(im_path)
    im_ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# #     value = (35, 35)
# #     blurred = cv2.GaussianBlur(res, value, 0)
    res = cv2.resize(im_ycrcb,(200, 200), interpolation=cv2.INTER_CUBIC)
    
    res = np.swapaxes(res, 0, 2)
    res = np.swapaxes(res, 1, 2)
    res = res[np.newaxis, :]
#     print res[0][1]
#     break
    
    mod.forward(Batch([mx.nd.array(res)]), is_train=False)
    fs = mod.get_outputs()[0].asnumpy()[0]
#     break
    out.append(fs)
    labels.append(ges_to_num[state])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [11]:
labels = np.array(labels)
out = np.array(out)

In [12]:
out.shape, labels.shape

((300, 11221), (300,))

In [13]:
data1 = np.ones((out.shape[0], out.shape[1]+1))
data1.shape

(300, 11222)

In [14]:
data1[:,:-1], data1[:,-1] = out, labels

In [15]:
df = pd.DataFrame(data1)

In [16]:
df.to_csv('vectors.csv', index=False)